In [ ]:
import warnings
import agate
from pathlib import Path
from shutil import copyfile
import os
import csv
from decimal import *

# Calculate maternal mortality, prenatal rates
# For rural/urban
# Border/non-border
def csv_calcuations(input_f, output_f, categories_dict, classification):
    try:
        with open(input_f, 'r') as inf, open(output_f, 'w') as outf:
            outcsv = csv.writer(outf, delimiter=',')
            
            # Skip header
            has_header = csv.Sniffer().has_header(inf.read(20048))
            inf.seek(0)
            incsv = csv.reader(inf)
            if has_header:
                next(incsv)
            
            for row in incsv:
                if '10-14' in input_f:
                    categories = ['count', 'births', 'deaths', 'prenatal_eligible', 'prenatal', 'race_female_total', 'race_female_white', 'race_female_black', 'race_female_hispanic', 'race_female_other']
                else:
                    categories = ['count', 'prenatal', 'prenatal_eligible', 'births', 'deaths']
                
                cat_loc = 3
                classify_row = row[2]
                
                if row[cat_loc]:
                    for category in categories:
                        if category is not 'count':
                            dict_check_add(category, classify_row, row[cat_loc], categories_dict, classification, row)
                            cat_loc += 1
                        else:
                            dict_check_add(category, classify_row, 1, categories_dict, classification, row)           

            # Create csv
            if '10-14' in input_f:
                csv_header = ['category', 'count', 'births', 'deaths', 'prenatal_eligible', 'prenatal', 'race_female_white', 'race_female_black', 'race_female_hispanic', 'race_female_other', 'race_female_white_perc', 'race_female_black_perc', 'race_female_hispanic_perc', 'race_female_other_perc', 'mortality_rate', 'prenatal_rate']
            else: 
                csv_header = ['category', 'count', 'births', 'deaths', 'prenatal_eligible', 'prenatal', 'mortality_rate', 'prenatal_rate']
            outcsv.writerow(csv_header)

            for key, value in categories_dict.items():
                mortality_rate = round((value['deaths'] / value['births']) * 100000, 2)
                prenatal_rate = round((value['prenatal'] / value['prenatal_eligible']) * 100, 2)
                
                if '10-14' in input_f:
                    races = ['white', 'black', 'hispanic', 'other']
                    races_perc = {}
                
                    for race in races:
                        percent = round(value['race_female_' + race] / value['race_female_total'] * 100, 2)
                        
                        races_perc[race] = percent
                
                print(key)
                print(value)
                
                print("Row created")
                if '10-14' in input_f:
                    outcsv.writerow([ key, value['count'], value['births'], value['deaths'], value['race_female_white'], value['race_female_black'], value['race_female_hispanic'], value['race_female_other'], value['prenatal_eligible'], value['prenatal'], races_perc['white'], races_perc['black'], races_perc['hispanic'], races_perc['other'], mortality_rate, prenatal_rate ])
                else:
                    outcsv.writerow([ key, value['count'], value['births'], value['deaths'], value['prenatal_eligible'], value['prenatal'], mortality_rate, prenatal_rate ])
    except ValueError:
        return False

In [ ]:
all_categories = ['urbanization', 'border', 'ob-hospitals']
# all_categories = ['urbanization', 'border']
# all_categories = ['urbanization', 'ob-hospitals']
# all_categories = ['ob-hospitals']

for a_category in all_categories:
    if a_category == 'ob-hospitals':
        years = '14'
    else:
        years = '10-14'
        
    g_table_input = '../edits/02-mortality-prenatal-' + a_category + '-' + years + '-population.csv'
    g_table_output = '../output/mortality-prenatal-' + a_category + '-' + years + '-population-split.csv'
    
    # Categories we want to divide by
    # These are the values in the spreadsheets
    categories_dict = {}
    
    if a_category is 'urbanization':
        categories_dict['Urban'] = {}
        categories_dict['Rural'] = {}
    elif a_category is 'border':
        categories_dict['Border'] = {}
        categories_dict['Non-Border'] = {}
    elif a_category == 'ob-hospitals':
        categories_dict['OB hospital(s)'] = {}
        categories_dict['No OB hospital(s)'] = {}
    categories_dict['Total'] = {}
    
    csv_calcuations(g_table_input, g_table_output, categories_dict, a_category)